In [369]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


In [370]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)


def cross_entropy(x):
    return -np.log(x)


def regularized_cross_entropy(layers, lam, x):
    loss = cross_entropy(x)
    for layer in layers:
        loss += lam * (np.linalg.norm(layer.get_weights()) ** 2)
    return loss


def leakyReLU(x, alpha=0.001):
    return np.where(x < 0, x * alpha, x)


def leakyReLU_derivative(x, alpha=0.01):
    return np.where(x < 0, alpha, 1)
def ReLU(Z):
        return np.maximum(Z, 0)
def ReLU_deriv(Z):
        return Z > 0


In [371]:

import numpy as np

class ConvLayer:
    def __init__(self, activation, padding="same", stride=(1, 1)):

        self.activation = activation
        self.padding = padding
        self.stride = stride
        self.w=None
        self.b=None

    def forward_prop(self,A_prev,W_conv,b_conv):
	    #forward prop convolutional 3D image, RGB image - color

        # Arg:
	    #    A_prev: contains the output of prev layer (m, h_prev, w_prev, c_prev)
	    #    W_conv: filter for the convolution (kh, kw, c_prev, c_new)
	    #    b_conv: biases (1, 1, 1, c_new)
	    #    padding: string ‘same’, or ‘valid’
	    #    stride: tuple (sh, sw)
        # Return: padded convolved images RGB np.array
        m, h_prev, w_prev, c_prev = A_prev.shape
        self.w=W_conv
        self.b=b_conv
        k_h, k_w, c_prev, c_new = self.w.shape

        s_h, s_w = self.stride
        if self.padding == 'valid':
            p_h = 0
            p_w = 0
        if self.padding == 'same':
            p_h = np.ceil(((s_h*h_prev) - s_h + k_h - h_prev) / 2)
            p_h = int(p_h)
            p_w = np.ceil(((s_w*w_prev) - s_w + k_w - w_prev) / 2)
            p_w = int(p_w)
        A_prev = np.pad(A_prev, [(0, 0), (p_h, p_h), (p_w, p_w), (0, 0)],
                        mode='constant', constant_values=0)

        out_h = int(((h_prev - k_h + (2*p_h)) / (self.stride[0])) + 1)
        out_w = int(((w_prev - k_w + (2*p_w)) / (self.stride[1])) + 1)
        output_conv = np.zeros((m, out_h, out_w, c_new))
        m_A_prev = np.arange(0, m)

        for i in range(out_h):
            for j in range(out_w):
                for f in range(c_new):
                    output_conv[m_A_prev, i, j, f] = self.activation((
                        np.sum(np.multiply(
                            A_prev[
                                m_A_prev,
                                i*(self.stride[0]):k_h+(i*(self.stride[0])),
                                j*(self.stride[1]):k_w+(j*(self.stride[1]))],
                            self.w[:, :, :, f]), axis=(1, 2, 3))) + self.b[0, 0, 0, f])


        return output_conv

    def backward_prop(self,dZ, A_prev):
	    #back prop convolutional 3D image, RGB image - color
	    # Arg:
	    #    dZ: containing the partial derivatives (m, h_new, w_new, c_new)
	    #    A_prev: contains the output of prev layer (m, h_prev, w_prev, c_prev)
	    #    W: filter for the convolution (kh, kw, c_prev, c_new)
	    #    b: biases (1, 1, 1, c_new)
	    #    padding: string ‘same’, or ‘valid’
	    #    stride: tuple (sh, sw)
	    # Returns: parcial dev prev layer (dA_prev), kernels (dW), biases (db)
        k_h, k_w, c_prev, c_new = self.w.shape
        _, h_new, w_new, c_new = dZ.shape
        m, h_x, w_x, c_prev = A_prev.shape
        s_h, s_w = self.stride
        x = A_prev
        if self.padding == 'valid':
            p_h = 0
            p_w = 0

        if self.padding == 'same':
            p_h = np.ceil(((s_h*h_x) - s_h + k_h - h_x) / 2)
            p_h = int(p_h)
            p_w = np.ceil(((s_w*w_x) - s_w + k_w - w_x) / 2)
            p_w = int(p_w)

        db = np.sum(dZ, axis=(0, 1, 2), keepdims=True)
        x_padded = np.pad(x, [(0, 0), (p_h, p_h), (p_w, p_w), (0, 0)],
                            mode='constant', constant_values=0)
        dW = np.zeros_like(self.w)
        dx = np.zeros(x_padded.shape)
        m_i = np.arange(m)
        for i in range(m):
            for h in range(h_new):
                for w in range(w_new):
                    for f in range(c_new):
                        dx[i,
                            h*(self.stride[0]):(h*(self.stride[0]))+k_h,
                            w*(self.stride[1]):(w*(self.stride[1]))+k_w,
                            :] += dZ[i, h, w, f] * self.w[:, :, :, f]
                        dW[:, :, :, f] += x_padded[i,
                                                h*(self.stride[0]):(h*(self.stride[0]))+k_h,
                                                w*(self.stride[1]):(w*(self.stride[1]))+k_w,
                                                :] * dZ[i, h, w, f]
        if self.padding == 'same':
            dx = dx[:, p_h:-p_h, p_w:-p_w, :]
        else:
            dx = dx
        return dx, dW, db


In [372]:
class Pooling:
    def __init__(self, kernel_shape,stride,mode='max'):
        self.kernel_shape = kernel_shape
        self.stride=stride
        self.mode=mode

    def forward_prop(self,A_prev):
        #pool forward prop convolutional 3D image, RGB image - color
        # Arg:
        # A_prev: contains the output of prev layer (m, h_prev, w_prev, c_prev)
        # W: filter for the convolution (kh, kw)
        # stride: tuple (sh, sw)
        # mode: indicates if max or avg
        # Return: output of the pooling layer
        m, h_prev, w_prev, c_prev = A_prev.shape
        k_h = self.kernel_shape[0]
        k_w = self.kernel_shape[1]

        out_h = int(((h_prev - k_h) / (self.stride[0])) + 1)
        out_w = int(((w_prev - k_w) / (self.stride[1])) + 1)
        output_conv = np.zeros((m, out_h, out_w, c_prev))
        m_A_prev = np.arange(0, m)
        for i in range(out_h):
            for j in range(out_w):
                if self.mode == 'max':
                    output_conv[m_A_prev, i, j] = np.max(
                        A_prev[m_A_prev,
                            i*(self.stride[0]):k_h+(i*(self.stride[0])),
                            j*(self.stride[1]):k_w+(j*(self.stride[1]))], axis=(1, 2))
                if self.mode == 'avg':
                    output_conv[m_A_prev, i, j] = np.mean(
                        A_prev[
                            m_A_prev,
                            i*(self.stride[0]):k_h+(i*(self.stride[0])),
                            j*(self.stride[1]):k_w+(j*(self.stride[1]))], axis=(1, 2))
        return output_conv
    def backward_prop(self,dA, A_prev):
    # back prop convolutional 3D image, RGB image - color
    # Arg:
    #    dA: containing the partial derivatives (m, h_new, w_new, c_new)
    #    A_prev: contains the output of prev layer (m, h_prev, w_prev, c)
    #    kernel.shape: filter dimensions tupple (kh, kw)
    #    stride: tuple (sh, sw)
    #    mode: max or avg
    # Returns: parcial dev prev layer (dA_prev)
        k_h = self.kernel_shape[0]
        k_w=self.kernel_shape[1]
        m, h_new, w_new, c_new = dA.shape
        m, h_x, w_x, c_prev = A_prev.shape
        s_h, s_w = self.stride

        dx = np.zeros_like(A_prev)

        for i in range(m):
            for h in range(h_new):
                for w in range(w_new):
                    for f in range(c_new):
                        if self.mode == 'max':
                            tmp = A_prev[i, h*s_h:k_h+(h*s_h),
                                        w*s_w:k_w+(w*s_w), f]
                            mask = (tmp == np.max(tmp))
                            dx[i,
                            h*(s_h):(h*(s_h))+k_h,
                            w*(s_w):(w*(s_w))+k_w,
                            f] += dA[i, h, w, f] * mask
                        if self.mode == 'avg':
                            dx[i,
                            h*(s_h):(h*(s_h))+k_h,
                            w*(s_w):(w*(s_w))+k_w,
                            f] += (dA[i, h, w, f])/k_h/k_w

        return dx



In [373]:
class Reshape_layer:
    def __init__(self):
        self.input_shape=None
    def forward_prop(self,X):
        self.input_shape=X.shape
        n_sample,_,_,_=X.shape
        return X.reshape(-1,n_sample)
    def backward_prop(self,X):
        return X.reshape(self.input_shape)

In [374]:
class fullycolected_layer:
    def __init__(self):
        self.w1=None
        self.b1=None
        self.w2=None
        self.b2=None
        self.w3=None
        self.b3=None
    def ReLU(self,Z):
        return np.maximum(Z, 0)
    def softmax(self,Z):
        A = np.exp(Z) / sum(np.exp(Z))
        return A
    def forward_prop(self, X,w1,b1,w2,b2,w3,b3):
        self.w1=w1
        self.b1=b1
        self.w2=w2
        self.b2=b2
        self.w3=w3
        self.b3=b3
        Z1 = w1.dot(X) + b1
        A1 = self.ReLU(Z1)
        Z2 = w2.dot(A1) + b2
        A2 = self.ReLU(Z2)
        Z3 = w3.dot(A2) + b3
        A3 = self.softmax(Z3)
        return Z1, A1, Z2, A2, Z3, A3
    def ReLU_deriv(self,Z):
        return Z > 0
    def one_hot(self,Y):
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        one_hot_Y = one_hot_Y.T
        return one_hot_Y
    def backward_prop(self,Z1, A1, Z2, A2, Z3, A3,X, Y):
        m=Y.size
        one_hot_Y = self.one_hot(Y)
        dZ3 = A3 - one_hot_Y
        dW3 = 1 / m * dZ3.dot(A2.T)
        db3 = 1 / m * np.sum(dZ3)
        dZ2 = self.w3.T.dot(dZ3) * self.ReLU_deriv(Z2)
        dW2 = 1 / m * dZ2.dot(A1.T)
        db2 = 1 / m * np.sum(dZ2)
        dZ1 = self.w2.T.dot(dZ2) * self.ReLU_deriv(Z1)
        dW1 = 1 / m * dZ1.dot(X.T)
        db1 = 1 / m * np.sum(dZ1)
        d_inp=self.w1.T.dot(dZ1)
        return dW1, db1, dW2, db2, dW3, db3, d_inp
    def update_params(self, dW1, db1, dW2, db2, dW3, db3, alpha):
        W1 = self.w1 - alpha * dW1
        b1 = self.b1 - alpha * db1
        W2 = self.w2 - alpha * dW2
        b2 = self.b2 - alpha * db2
        W3 = self.w3 - alpha * dW3
        b3 = self.b3 - alpha * db3
        return W1,b1,W2,b2,W3,b3

In [375]:
data=pd.read_csv('D:\MachineLearning\CNN\Data_skincancer\hmnist_28_28_RGB.csv')


In [376]:
print(data.shape)

(10015, 2353)


In [377]:
X=data.iloc[:,0:-1].values
y=data.iloc[:,-1].values

In [378]:
from sklearn.model_selection import train_test_split

In [379]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [380]:
X_train=X_train.reshape(-1,28,28,3)
X_test=X_test.reshape(-1,28,28,3)

In [381]:
print(X_train.shape)
print(X_test.shape)

(8012, 28, 28, 3)
(2003, 28, 28, 3)


In [382]:
X_train_scaled=X_train/255-0.5
X_test_scaled=X_test/255-0.5

In [383]:
print(X_train.shape)

(8012, 28, 28, 3)


In [384]:
conv_layer=ConvLayer(padding='valid',activation=ReLU)
max_pool=Pooling(kernel_shape=(3,3),stride=(3,3),mode='max')
reshape=Reshape_layer()
full_layer=fullycolected_layer()


In [385]:
class trainning:
    def __init__(self,conv_layer,pool,reshape,fully):
         self.conv_layer=conv_layer
         self.pool=pool
         self.full_layer=fully
         self.reshape=reshape
         self.w1=None
         self.b1=None
         self.w2=None
         self.b2=None
         self.w3=None
         self.b3=None
         self.w_conv=None
         self.b_conv=None

    def __init__conv(self,filter_size,num_filters):
        w_conv=np.random.rand(filter_size, filter_size, 3, num_filters)
        b_conv = np.zeros((1, 1, 1, num_filters))
        return w_conv,b_conv

    def init_fully(self,X):
        h,w=X.shape
        W1 = np.random.rand(7,h) - 0.5
        b1 = np.random.rand(7, 1) - 0.5
        W2 = np.random.rand(7, 7) - 0.5
        b2 = np.random.rand(7, 1) - 0.5
        W3 = np.random.rand(7, 7) - 0.5
        b3 = np.random.rand(7, 1) - 0.5
        return W1, b1, W2, b2, W3,b3
    def get_predictions(self,A2):
            return np.argmax(A2, 0)

    def get_accuracy(self,predictions, Y):
        print(predictions, Y)
        return np.sum(predictions == Y) / Y.size

    def gradient_descent(self,X, Y, epochs=10, lr=0.01):
        self.w_conv,self.b_conv= self.__init__conv(3,7)

        for i in range(epochs):
            print('epochs', i)
            print('forward')
            conv_output = self.conv_layer.forward_prop(X,self.w_conv,self.b_conv)
            print('conv_out',conv_output.shape)
            pooled_output = self.pool.forward_prop(conv_output)
            print('pool_out',pooled_output.shape)
            flattened_output = self.reshape.forward_prop(pooled_output)
            print('flat_out',flattened_output.shape)
            if i==0:
                 self.w1,self.b1,self.w2,self.b2,self.w3,self.b3=self.init_fully(flattened_output)
            print(self.w1.max())
            print(self.w2.max())
            Z1, A1, Z2, A2, Z3, A3 = self.full_layer.forward_prop(flattened_output,self.w1,self.b1,self.w2,self.b2,self.w3,self.b3)
            print('backward')
            dw1, db1, dw2, db2, dw3,db3,dZ1 = self.full_layer.backward_prop(Z1, A1, Z2, A2, Z3, A3, flattened_output, Y)
            dZ1_reshape=self.reshape.backward_prop(dZ1)
            print('dZ1_reshape',dZ1_reshape.shape)
            dl_dmax_pool=self.pool.backward_prop(dZ1_reshape,conv_output)
            print('dmax_pool',dl_dmax_pool.shape)
            dZ=ReLU_deriv(dl_dmax_pool)
            print('dz',dZ.shape)
            _,dw_conv,db_conv=conv_layer.backward_prop(dZ,X_train_scaled)
            print('dw_conv',dw_conv.shape)
            print('db_conv',db_conv.shape)
            self.w1 = self.w1 - lr * dw1
            self.b1 = self.b1 - lr * db1
            self.w2 = self.w2 - lr * dw2
            self.b2 = self.b2 - lr * db2
            self.w3 = self.w3 - lr * dw3
            self.b3 = self.b3 - lr * db3
            self.w_conv=self.w_conv-lr*dw_conv
            self.b_conv=self.b_conv-lr*db_conv
            predictions = self.get_predictions(A3)
            print(self.get_accuracy(predictions, Y))
        return self.w1, self.b1, self.w2, self.b2,self.w3,self.b3,self.w_conv,self.b_conv

In [386]:
conv_layer=ConvLayer(padding='valid',activation=ReLU)
max_pool=Pooling(kernel_shape=(2,2),stride=(2,2),mode='max')
reshape=Reshape_layer()
full_layer=fullycolected_layer()

In [387]:
trainning_cnn=trainning(conv_layer,max_pool,reshape,full_layer)

In [ ]:
W1, b1, W2, b2,W3,b3,w_conv=trainning_cnn.gradient_descent(X_train_scaled,y_train)